## Python, SQL and defaultdict

In this notebook we're going to put a bunch of pieces together. 

* SQL: We'll build an "in memory" DB and write some complicated queries against it.
* `defaultdict`: building on the recorded lecture, we'll take the results of those complicated queries and store them in a sensible way in a `defaultdict`. We'll also practice writing these out to a file. 

This code contains a number of the techniques you need to do the _second_ part of the Wedge assignment. 

We start by reading in the file as before. Note the import of our `wedge_helper` library, which is stored in a file with that same name and the extension `py`. As before, open that file up (maybe in Spyder?) so that you can see what's inside.

In [ ]:
import sqlite3
from pprint import pprint
from collections import defaultdict
from wedge_helper import *

input_file = "OwnerTransactions_30.txt"

# Let's just open the file and read the first few lines to the screen.
with open(input_file,'r',encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile.readlines()) :
        print(line.strip().split("\t"))
        if idx > 3 :
            break

Let's do our standard open the DB in memory and create a cursor.

In [ ]:
db = sqlite3.connect(':memory:') # SQLite makes it easy to build in-memory DBs
cur = db.cursor()

init_db(cur) # take a look at the .py file to see what happened here.

In [ ]:
with open(input_file,'r') as ifile :
    populate_db(db,ifile,delimiter="\t",limit=None)

Because we're using functions from the helper file, what we just did looks deceptively simple. There's a full `transactions` table now stored in a DB in memory. And `cur` holds the connection to that DB.

In [ ]:
cur.execute('SELECT * FROM transactions LIMIT 3')
cur.fetchall()

Notice how the query results come back. What sort of data type is it? What's the "inner" data type?

Again we'll run a "simple" query and print out the results in a semi-pretty fashion. Check out [this page](https://docs.python.org/3.6/library/string.html#format-specification-mini-language) to learn more about the formatting tricks.

In [ ]:
result = cur.execute('''SELECT card_no,
                               date(datetime) as date,
                               sum(total) AS spend
                        FROM transactions
                        WHERE trans_type = "I"
                        GROUP BY card_no, date
                        ''')

for idx,row in enumerate(result) :
    print("On {1}, card_no = {0} spent {2:,.2f}.".format(row[0],row[1],row[2]))
    if idx > 20 :
        break

Obviously, in the real world you wouldn't want to just print this stuff to the screen, you'd store it. A dictionary, with keys of date and owner number, is a natural way to do that. As in the lecture, we'll do this first in a plain dictionary.

In [ ]:
date_owner_spend = dict()

for idx,row in enumerate(result) :
    owner, date, spend = row
    spend = round(spend,2)
    
    if owner not in date_owner_spend :
        date_owner_spend[owner] = dict()
        date_owner_spend[owner][date] = spend
    else : 
        if date not in date_owner_spend[owner] :
            date_owner_spend[owner][date] = spend
        else :
            date_owner_spend[owner][date] += spend

In [ ]:
# Let's look at one owner's spend
pprint(date_owner_spend[18736])

Let's do the same thing, but with `defaultdict`.

In [ ]:
date_owner_spend_2 = defaultdict(lambda: defaultdict(float)) 
for idx,row in enumerate(result) :
    owner, date, spend = row
    spend = round(spend,2)
    
    date_owner_spend_2[owner][date] += spend
    

In [ ]:
pprint(date_owner_spend[18736])

Now let's do a bigger query, like the one from last class. Let's get sales, transactions and items by department, year and month.

In [ ]:
result = cur.execute('''SELECT department,
                               substr(date(datetime),1,4) AS year,
                               substr(date(datetime),6,2) AS month,
                               sum(total) AS spend,
                               count(distinct(date(datetime) || register_no ||
                                       emp_no || trans_no)) as Transactions,
                               sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
                               FROM transactions
                               WHERE department != 0 and
                                    department != 15 and
                                    trans_status != 'M' and
                                    trans_status != 'C' and
                                    trans_status != 'J' and
                                   (trans_status = '' or 
                                    trans_status = ' ' or 
                                    trans_status = 'V' or 
                                    trans_status = 'R')
                          GROUP BY department, year, month
                          ORDER BY year, month''')

In [ ]:
dept_ym = defaultdict(lambda: # dept_no
                      defaultdict(lambda: # year 
                                  defaultdict(lambda: # month
                                              defaultdict(float)))) # #field name and value
                      
for row in result :
    dept, year, month, spend, trans, items = row
    spend = round(spend,2)

    dept_ym[dept][year][month]['spend'] += spend
    dept_ym[dept][year][month]['trans'] += trans
    dept_ym[dept][year][month]['items'] += items

Play around with this dictionary a bit to make sure you understand the structure.

Sometimes having the data in memory is all you need. Other times you're going to want to write it out to a file to work with further. Let's write out this dictionary.

In [ ]:
header = ["dept_no","year","month","spend","transactions","items"]

with open("dept_year_month.txt",'w') as ofile :
    ofile.write("\t".join(header) + "\n")
    
    for dept in dept_ym :
        for year in dept_ym[dept] :
            for month in dept_ym[dept][year] :
                row = [dept,year,month,
                      dept_ym[dept][year][month]['spend'],
                      dept_ym[dept][year][month]['trans'],
                      dept_ym[dept][year][month]['items']]
                
                ofile.write("\t".join([str(item) for item in row]) + "\n")
    

---

Now it's your turn. Write a query that builds the first task in Wedge Part 2: sales (and transactions and items) by date by hour. We haven't talked too much about how you get "hour" out of one of these transaction tables, so that's going to require some trial and error. One option for building queries: redo this notebook where the db is _not_ `:memory:` but where you actually give it a name. Then it will be on your computer, you can open your SQLite Browser and play around there. 

In [ ]:
result = cur.execute('''
                        
                        -- Your query here

                    ''')

Now store it in a `defaultdict` and write it out in the subsequent cell.

In [ ]:
# Store query results in a default dict

In [ ]:
# write out your default dict

Run the cell below as a best practice, but the DB will close if you quit the notebook. If it's an in-memory DB then it's also gone!

In [ ]:
db.close()